# // Getting Team-Level Statistics
___
In our [first notebook](https://github.com/dodgemcintosh/FFML/blob/master/1-Getting-NFL-Data.ipynb), we used several web scraping techniques to bring in individual player statistics from each game in the 2018 season.

In this notebook, we're going to be bringing in some team-level stats from both the defensive and offensive sides of the ball that should prove to be useful features for our predictive modeling. Some of those features are:
- Defensive formations
- Offensive play distribution
- Weather conditions
- And more!

While we're going to continue to use our good friends resources over at [FantasyPros](https://www.fantasypros.com/nfl/), we'll also be bringing these other friends to the party:
- [The Football Database](https://www.footballdb.com/)
- [Fantasy Sports Doctors](http://fantasysportdrs.com/)
- [IDPGuru](http://www.idpguru.com/)
- [FantasyGuru](https://www.fantasyguru.com/)

In [1]:
# Let's import everything we're going to need

import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
%matplotlib inline

## Going to start with defensive schemes and tendencies
- After searching around quite a bit, I decided that the most robust resource will actually require me to input some of the data by hand; the resulting `.csv` is explored briefly below:

In [6]:
df_schemes = pd.read_csv('./data/nfl_defensive_schemes_2018.csv')
df_schemes.head()

,team,division,head_coach,def_coordinator,base,tendency
0,ARI,NFC West,Steve Wilks,Al Holcomb,4-3,Zone-heavy mix
1,ATL,NFC South,Dan Quinn,Marquand Manuel,4-3,Cover-3 (Zone)
2,BAL,AFC North,John Harbaugh,Don Martindale,3-4,Mix of Man/Zone
3,BUF,AFC East,Sean McDermott,Leslie Frazier,4-3,Zone
4,CAR,NFC South,Ron Rivera,Eric Washington,4-3,Cover-3 (Zone)


In [8]:
# Doing one quick spot check to make sure that all divisions are represented equally
df_schemes.division.value_counts()

AFC South    4
AFC West     4
NFC North    4
NFC East     4
AFC East     4
AFC North    4
NFC West     4
NFC South    4
Name: division, dtype: int64